# Finding eigenmaps with Different Methods

## Import Needed Libraries

In [32]:
import os
import sys
import mc3
import pickle
import starry
import shutil
import subprocess
import progressbar
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd

In [33]:
# Taurex imports
import taurex
from taurex import chemistry
from taurex import planet
from taurex import stellar
from taurex import model
from taurex import pressure
from taurex import temperature
from taurex import cache
from taurex import contributions
from taurex import optimizer
# This import is explicit because it's not included in taurex.temperature. Bug?
from taurex.data.profiles.temperature.temparray import TemperatureArray

# Taurex is a bit...talkative
import taurex.log
taurex.log.disableLogging()



In [34]:
# Directory structure
__file__ = os.path.abspath('') # __file__ is not defined in jupyter

maindir    = os.path.dirname(os.path.realpath(__file__))
libdir     = os.path.join(maindir, 'lib2')
moddir     = os.path.join(libdir,  'modules')
ratedir    = os.path.join(moddir,  'rate')
transitdir = os.path.join(moddir, 'transit')

In [35]:
# Lib imports
sys.path.append(libdir)
print(sys.path)
from lib2 import constants
from lib2 import cf
from lib2 import atm
from lib2 import my_pca as pca
from lib2 import star_eigen as eigen
from lib2 import model
from lib2 import star_plots as plots
from lib2 import mkcfg
from lib2 import star_utils as utils
from lib2 import constants   as c
from lib2 import star_fitclass    as fc
from lib2 import taurexclass as trc

starry.config.quiet = True

['/Library/Frameworks/Python.framework/Versions/3.9/lib/python39.zip', '/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9', '/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/lib-dynload', '', '/Users/a.j.devaux/Downloads/Research/stellar_null_space/ryan_theresa/jupyter_theresa/.venv/lib/python3.9/site-packages', '/Users/a.j.devaux/Downloads/Research/stellar_null_space/ryan_theresa/jupyter_theresa/.venv/lib/python3.9/site-packages/mc3/lib/', '/Users/a.j.devaux/Downloads/Research/stellar_null_space/ryan_theresa/jupyter_theresa/.venv/lib/python3.9/site-packages/mc3/lib', '/Users/a.j.devaux/Downloads/Research/stellar_null_space/ryan_theresa/lib2', '/Users/a.j.devaux/Downloads/Research/stellar_null_space/ryan_theresa/jupyter_theresa/lib2/modules/rate', '/Users/a.j.devaux/Downloads/Research/stellar_null_space/ryan_theresa/lib2']


## Setting up PCA/Eigenmap in Notebook

In [36]:
import numpy as np
import sklearn
import sklearn.decomposition

def pca(arr, method='pca', ncomp=None):
    """
    Runs principle component analysis on the input array.

    Arguments
    ---------
    arr: 2D array
        (m, n) array, where n is the size of the dataset (e.g., times
        in an observation) and m is the number of vectors

    Returns
    -------
    evalues: 1D array
        array of eigenvalues of size n

    evectors: 2D array
        (m, m) array of sorted eigenvectors

    proj: 2D array
        (m, n) array of data projected in the new space

    Notes
    -----
    See https://glowingpython.blogspot.com/2011/07/pca-and-image-compression-with-numpy.html
    """
    nt = arr.shape[1]
    print("We are using the right pca!")
    
    if method == 'pca':
        arr = arr.T
        # Subtract the mean
        m = (arr - np.mean(arr.T, axis=1)).T
        #m = arr
        # Compute eigenvalues
        evalues, evectors = np.linalg.eig(np.cov(m))
        # Sort descending
        idx = np.argsort(evalues)[::-1]
        evalues  = evalues[   idx]
        evectors = evectors[:,idx]
        # Calculate projection of the data in the new space
        proj = np.dot(evectors.T, m)

    elif method == 'tsvd':
        tpca = sklearn.decomposition.TruncatedSVD(n_components=ncomp)
        tpca.fit(arr.T)
        evalues  = tpca.explained_variance_
        evectors = tpca.components_
        proj = np.zeros((ncomp, nt))
        # print(evectors.shape)
        # print(evectors[0].shape)
        # print(arr.T.shape)
        # test_1 = evectors[0] * arr.T
        # print(test_1.shape)
        # print(np.sum(test_1, axis=1).shape)

        for i in range(ncomp):
        # for i in range(0,2*ncomp,2):
            proj[i] = np.sum(evectors[i] * arr.T, axis=1)

        evectors = evectors.T
            
        
    return evalues, evectors, proj


In [37]:
import numpy as np
import scipy.constants as sc

def mkcurves(star, t, lmax, y00, ncurves=None, method='pca', negative = True, remove_y00 = True, all_curves = False):
    """
    Generates light curves from a star+planet system at times t,
    for positive and negative spherical harmonics with l up to lmax.

    Arguments
    ---------
    system: object
        A starry system object, initialized with a star and a planet

    t: 1D array
        Array of times at which to calculate eigencurves

    lmax: integer
        Maximum l to use in spherical harmonic maps

    y00: 1D array
        Light curve of a normalized, uniform map

    Returns
    -------
    eigeny: 2D array
        nharm x ny array of y coefficients for each harmonic. nharm is
        the number of harmonics, including positive and negative versions
        and excluding Y00. That is, 2 * ((lmax + 1)**2 - 1). ny is the
        number of y coefficients to describe a harmonic with degree lmax.
        That is, (lmax + 1)**2.

    evalues: 1D array
        nharm length array of eigenvalues

    evectors: 2D array
        nharm x nt array of normalized (unit) eigenvectors

    proj: 2D array
        nharm x nt array of the data projected in the new space (the PCA
        "eigencurves"). The imaginary part is discarded, if nonzero.
    """    
    # star   = system.bodies[0]
    # planet = system.bodies[1]

    nt = len(t)
    print("We are using the right mkcurves!")

    thet = np.linspace(0, 360, nt)

    if negative:
    
        # Create harmonic maps of the planet, excluding Y00
        # (lmax**2 maps, plus a negative version for all but Y00)
        nharm = ((lmax + 1)**2 - 1) *2 #trying to make it only 15 instead of 30
        lcs = np.zeros((nharm, nt))
        ind = 0
        for i, l in enumerate(range(1, lmax + 1)):
            for j, m in enumerate(range(-l, l + 1)):  
                star.map[l, m] =  1.0 #update why?
                lcs[ind] = star.map.flux(theta=thet).eval()
        

                star.map[l, m] = -1.0 #update why? is every second row negative version???
                lcs[ind+1] = star.map.flux(theta=thet).eval()


                star.map[l, m] = 0.0 #how the map started
                ind += 2
       

        print(lcs.shape)
    else:
         # Create harmonic maps of the planet, excluding Y00
        # (lmax**2 maps, plus a negative version for all but Y00)
        nharm = ((lmax + 1)**2 - 1)  #trying to make it only 15 instead of 30
        lcs = np.zeros((nharm, nt))
        ind = 0
        for i, l in enumerate(range(1, lmax + 1)):
            for j, m in enumerate(range(-l, l + 1)):  
                star.map[l, m] =  1.0 #update why?
                lcs[ind] = star.map.flux(theta=thet).eval()


                star.map[l, m] = 0.0 #how the map started
                ind += 1
       

        print(lcs.shape)


    if remove_y00:
        # Subtact uniform map contribution (starry includes this in all
        # light curves)
        lcs -= y00 # will be zero for a star-only system

    # np.savetxt("lcs_test.txt", lcs, delimiter=",")
    # return None #need to fix this
            
    # Run PCA to determine orthogonal light curves
    if ncurves is None:
        ncurves = nharm
        
    evalues, evectors, proj = pca(lcs, method=method, ncomp=ncurves)

    print(evectors.shape)

    # Discard imaginary part of eigencurves to appease numpy
    proj = np.real(proj)


    if negative:

        if method == 'pca' and all_curves:
            ncurves = ncurves * 2

        # Convert orthogonal light curves into maps        
        eigeny = np.zeros((ncurves, (lmax + 1)**2))
        eigeny[:,0] = 1.0 # Y00 = 1 for all maps

        for j in range(ncurves):
            yi  = 1
            shi = 0
            for l in range(1, lmax + 1):
                for m in range(-l, l + 1):
                    # (ok because evectors has only been sorted along
                    #  one dimension)
                    eigeny[j,yi] = evectors.T[j,shi] - evectors.T[j,shi+1]
                    yi  += 1
                    shi += 2
                    # shi+= 1

    else:
        # Convert orthogonal light curves into maps        
        eigeny = np.zeros((ncurves, (lmax + 1)**2))
        eigeny[:,0] = 1.0 # Y00 = 1 for all maps

        for j in range(ncurves):
            yi  = 1
            shi = 0
            for l in range(1, lmax + 1):
                for m in range(-l, l + 1):
                    # (ok because evectors has only been sorted along
                    #  one dimension)
                    eigeny[j,yi] = evectors.T[j,shi] #- evectors.T[j,shi+1]
                    yi  += 1
                    shi += 1

    return eigeny, evalues, evectors, proj, lcs

In [38]:
def emaps(star, eigeny, outdir, proj='ortho'):
    ncurves, ny = eigeny.shape
   
    # thet = np.linspace(0, 360, ny)

    if proj == 'ortho':
        extent = (-90, 90, -90, 90)
        fname = 'emaps-ecl.png'
    elif proj == 'rect':
        extent = (-180, 180, -90, 90)
        fname = 'emaps-rect.png'
    elif proj == 'moll':
        extent = (-180, 180, -90, 90)
        fname = 'emaps-moll.png'

    lmax = int(ny**0.5 - 1)

    ncols = int(np.sqrt(ncurves) // 1)
    nrows = int(ncurves // ncols + (ncurves % ncols != 0))
    npane = ncols * nrows

    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, squeeze=False,
                             sharex=True, sharey=True)
    
    for j in range(ncurves):
        star.map[1:,:] = 0

        xloc = j %  ncols
        yloc = j // ncols
        ax = axes[yloc, xloc]
        
        yi = 1
        for l in range(1, lmax + 1):
            for m in range(-l, l + 1):
                star.map[l, m] = eigeny[j, yi]
                yi += 1

        # print(f"The {j} index eigen map's flux is {star.map.flux(theta=thet).eval()}")
        
        ax.imshow(star.map.render(theta=0, projection=proj).eval(),
                  origin="lower",
                  cmap="plasma",
                  extent=extent)
        
        # star.map.show(theta=np.linspace(0, 360, 50))
    

        # Axes are wrong for non-rectangular projections
        if proj == 'ortho' or proj == 'moll':
            ax.axis('off')

    # Empty subplots
    for j in range(ncurves, npane):
        xloc = j %  ncols
        yloc = j // ncols
        ax = axes[yloc, xloc]

        ax.axis('off')

    fig.tight_layout()
    plt.savefig(f"plots/{outdir}")
    plt.close(fig)

## Set-up

In [39]:
print(sys.argv)
cfile = "first_star.cfg"
dimen = "2D"

fit = fc.Fit()
print("Reading the configuration file.")
fit.read_config(cfile)
cfg = fit.cfg

print("Reading the data.")
fit.read_data()

lmax = cfg.twod.lmax

print("First star and planet objects.")
star = utils.initsystem(fit, lmax)

print("Finding Inclination of planet")
print(cfg.star.inc)

print("Finding Rank of Design Matrix")
A = star.map.design_matrix(theta=np.linspace(0,360,1000)) # how do I decide the angular phase???
# print(A.eval())

nsamples = cfg.twod.nsamples

# print(nsamples, lmax)

R = np.empty((nsamples, lmax))

for k in range(nsamples):
    
    R[k] = [
        np.linalg.matrix_rank(A.eval()[:, : (l + 1) ** 2]) for l in range(1, lmax + 1)
    ]
    # print(R[k])

R = np.median(R, axis=0)   

print(f"The design matrix is {R} for lmax = {lmax}")

# Uniform star object
star = utils.initsystem(fit, 1)

fit.sflux = star.map.flux(theta=fit.t.shape).eval()

print(f"The sflux is: {fit.sflux}")

print(f"The star map has flux {star.map.flux(theta=np.linspace(0,360,len(fit.t))).eval()}")

print(f"The time shape is {fit.t.shape}")

print(star.map.inc.eval())

fit.maps = []

fit.maps.append(fc.Map())
m = fit.maps[0]
m.ncurves = cfg.twod.ncurves
m.lmax    = cfg.twod.lmax
# m.wlmid = fit.wlmid[i]

if not os.path.isdir(cfg.outdir):
    os.mkdir(cfg.outdir)

m.subdir = 'filt{}'.format(1)
outdir = os.path.join(cfg.outdir, m.subdir)

if not os.path.isdir(os.path.join(cfg.outdir, m.subdir)):
    os.mkdir(os.path.join(cfg.outdir, m.subdir))

# New planet object with updated lmax
star = utils.initsystem(fit, m.lmax)

['/Users/a.j.devaux/Downloads/Research/stellar_null_space/ryan_theresa/jupyter_theresa/.venv/lib/python3.9/site-packages/ipykernel_launcher.py', '--f=/Users/a.j.devaux/Library/Jupyter/runtime/kernel-v36bb4512b08dc77f5b147222214acea8dde211bd3.json']
Reading the configuration file.
Reading the data.
First star and planet objects.
Finding Inclination of planet
72.0
Finding Rank of Design Matrix
The design matrix is [3. 5. 5.] for lmax = 3
The sflux is: [1.]
The star map has flux [1. 1. 1. ... 1. 1. 1.]
The time shape is (15937,)
72.0


## Running PCA's

In [78]:
def create_pcas():
    user_input = input("Do you want to create the PCAs or read from a file? (y/n)")

    return user_input.strip().lower() == "y"


dont_read_file = create_pcas()
    

In [75]:
# mkcurves(star, t, lmax, y00, ncurves=None, method='pca', negative = True, remove_y00 = True):
if dont_read_file:
    print("Making eigen values with tsvd and removing y00")
    tsvd_norm_eigeny, tsvd_norm_evalues, tsvd_norm_evectors, tsvd_norm_ecurves, tsvd_norm_lcs = \
    mkcurves(star, fit.t, m.lmax, fit.sflux,
                    ncurves=m.ncurves, method="tsvd", negative=True, remove_y00=True)
    
    np.savetxt("stored_pcas/tsvd_norm_eigeny.txt", tsvd_norm_eigeny)

else:
    try:
        tsvd_norm_eigeny = np.loadtxt("stored_pcas/tsvd_norm_eigeny.txt")
    except:
        print("PCA file not found, change read_file to True.")



Making eigen values with tsvd and removing y00
We are using the right mkcurves!
(30, 15937)
We are using the right pca!
(30, 15)


In [76]:
if dont_read_file:
    print("Making eigen values with tsvd and removing y00 again")
    tsvd_norm_eigeny2, tsvd_norm_evalues2, tsvd_norm_evectors2, tsvd_norm_ecurves2, tsvd_norm_lcs2 = \
    mkcurves(star, fit.t, m.lmax, fit.sflux,
                    ncurves=m.ncurves, method="tsvd", negative=True, remove_y00=True)
    
    np.savetxt("stored_pcas/tsvd_norm_eigeny2.txt", tsvd_norm_eigeny2)

else:
    try:
        tsvd_norm_eigeny2 = np.loadtxt("stored_pcas/tsvd_norm_eigeny2.txt")
    except:
        print("PCA file not found, change read_file to True.")
    

Making eigen values with tsvd and removing y00 again
We are using the right mkcurves!
(30, 15937)
We are using the right pca!
(30, 15)


In [77]:
if dont_read_file:
    print("Making eigen values with tsvd and removing y00 again")
    tsvd_norm_eigeny3, tsvd_norm_evalues3, tsvd_norm_evectors3, tsvd_norm_ecurves3, tsvd_norm_lcs3 = \
    mkcurves(star, fit.t, m.lmax, fit.sflux,
                    ncurves=m.ncurves, method="tsvd", negative=True, remove_y00=True)
    
    np.savetxt("stored_pcas/tsvd_norm_eigeny3.txt", tsvd_norm_eigeny3)

else:
    try:
        tsvd_norm_eigeny3 = np.loadtxt("stored_pcas/tsvd_norm_eigeny3.txt")
    except:
        print("PCA file not found, change read_file to True.")

Making eigen values with tsvd and removing y00 again
We are using the right mkcurves!
(30, 15937)
We are using the right pca!
(30, 15)


In [57]:
if dont_read_file:
    print("Making eigen values with tsvd and keeping y00")
    tsvd_y00_eigeny, tsvd_y00_evalues, tsvd_y00_evectors, tsvd_y00_ecurves, tsvd_y00_lcs = \
    mkcurves(star, fit.t, m.lmax, fit.sflux,
                    ncurves=m.ncurves, method="tsvd", negative=True, remove_y00=False)
    
    np.savetxt("stored_pcas/tsvd_y00_eigeny.txt", tsvd_y00_eigeny)

else:
    try:
        tsvd_y00_eigeny = np.loadtxt("stored_pcas/tsvd_y00_eigeny.txt")
    except:
        print("PCA file not found, change read_file to True.")


In [58]:
if dont_read_file:
    print("Making eigen values with tsvd and keeping y00 again")
    tsvd_y00_eigeny2, tsvd_y00_evalues2, tsvd_y00_evectors2, tsvd_y00_ecurves2, tsvd_y00_lcs2 = \
    mkcurves(star, fit.t, m.lmax, fit.sflux,
                    ncurves=m.ncurves, method="tsvd", negative=True, remove_y00=False)
    
    np.savetxt("stored_pcas/tsvd_y00_eigeny2.txt", tsvd_y00_eigeny2)

else:
    try:
        tsvd_y00_eigeny2 = np.loadtxt("stored_pcas/tsvd_y00_eigeny2.txt")
    except:
        print("PCA file not found, change read_file to True.")

In [59]:
if dont_read_file:
    print("Making eigen values with pca and removing y00")
    pca_norm_eigeny, pca_norm_evalues, pca_norm_evectors, pca_norm_ecurves, pca_norm_lcs = \
    mkcurves(star, fit.t, m.lmax, fit.sflux,
                    ncurves=m.ncurves, method="pca", negative=True, remove_y00=True)
    
    np.savetxt("stored_pcas/pca_norm_eigeny.txt", pca_norm_eigeny)

else:
    try:
        pca_norm_eigeny = np.loadtxt("stored_pcas/pca_norm_eigeny.txt")
    except:
        print("PCA file not found, change read_file to True.")   

In [60]:
if dont_read_file:
    print("Making eigen values with pca and removing y00 again")
    pca_norm_eigeny2, pca_norm_evalues2, pca_norm_evectors2, pca_norm_ecurves2, pca_norm_lcs2 = \
    mkcurves(star, fit.t, m.lmax, fit.sflux,
                    ncurves=m.ncurves, method="pca", negative=True, remove_y00=True)
    
    np.savetxt("stored_pcas/pca_norm_eigeny2.txt", pca_norm_eigeny2)

else:
    try:
        pca_norm_eigeny2 = np.loadtxt("stored_pcas/pca_norm_eigeny2.txt")
    except:
        print("PCA file not found, change read_file to True.")   

In [61]:
if dont_read_file:
    print("Making eigen values with pca and keeping y00")
    pca_y00_eigeny, pca_y00_evalues, pca_y00_evectors, pca_y00_ecurves, pca_y00_neg_lcs = \
    mkcurves(star, fit.t, m.lmax, fit.sflux,
                    ncurves=m.ncurves, method="pca", negative=True, remove_y00=False)
    
    np.savetxt("stored_pcas/pca_y00_eigeny.txt", pca_y00_eigeny)

else:
    try:
        pca_y00_eigeny = np.loadtxt("stored_pcas/pca_y00_eigeny.txt")
    except:
        print("PCA file not found, change read_file to True.")   

In [62]:
if dont_read_file:
    print("Making eigen values with pca and keeping y00 again")
    pca_y00_eigeny2, pca_y00_evalues2, pca_y00_evectors2, pca_y00_ecurves2, pca_y00_neg_lcs2 = \
    mkcurves(star, fit.t, m.lmax, fit.sflux,
                    ncurves=m.ncurves, method="pca", negative=True, remove_y00=False)

    np.savetxt("stored_pcas/pca_y00_eigeny2.txt", pca_y00_eigeny2)

else:
    try:
        pca_y00_eigeny2 = np.loadtxt("stored_pcas/pca_y00_eigeny2.txt")
    except:
        print("PCA file not found, change read_file to True.") 

In [63]:
if dont_read_file:
    print("Making eigen values with pca, removing y00, and no negative maps")
    pca_norm_pos_eigeny, pca_norm_pos_evalues, pca_norm_pos_evectors, pca_norm_pos_ecurves, pca_norm_pos_lcs = \
    mkcurves(star, fit.t, m.lmax, fit.sflux,
                    ncurves=m.ncurves, method="pca", negative=False, remove_y00=True)

    np.savetxt("stored_pcas/pca_norm_pos_eigeny.txt", pca_norm_pos_eigeny)

else:
    try:
        pca_norm_pos_eigeny = np.loadtxt("stored_pcas/pca_norm_pos_eigeny.txt")
    except:
        print("PCA file not found, change read_file to True.") 

In [64]:
if dont_read_file:
    print("Making eigen values with pca, keeping y00, and no negative maps")
    pca_y00_pos_eigeny, pca_y00_pos_evalues, pca_y00_pos_evectors, pca_y00_pos_ecurves, pca_y00_pos_lcs = \
    mkcurves(star, fit.t, m.lmax, fit.sflux,
                    ncurves=m.ncurves, method="pca", negative=False, remove_y00=False)
    
    np.savetxt("stored_pcas/pca_y00_pos_eigeny.txt", pca_y00_pos_eigeny)

else:
    try:
        pca_y00_pos_eigeny = np.loadtxt("stored_pcas/pca_y00_pos_eigeny.txt")
    except:
        print("PCA file not found, change read_file to True.") 

In [65]:
if dont_read_file:
    print("Making eigen values with pca, all 30 curves, and removing y00")
    pca30_norm_eigeny, pca30_norm_evalues, pca30_norm_evectors, pca30_norm_ecurves, pca30_norm_lcs = \
    mkcurves(star, fit.t, m.lmax, fit.sflux,
                    ncurves=m.ncurves, method="pca", negative=True, remove_y00=True, all_curves=True)
    
    np.savetxt("stored_pcas/pca30_norm_eigeny.txt", pca30_norm_eigeny)

else:
    try:
        pca30_norm_eigeny = np.loadtxt("stored_pcas/pca30_norm_eigeny.txt")
    except:
        print("PCA file not found, change read_file to True.") 

In [66]:
if dont_read_file:
    print("Making eigen values with pca, all 30 curves, and keeping y00")
    pca30_y00_eigeny, pca30_y00_evalues, pca30_y00_evectors, pca30_y00_ecurves, pca30_y00_neg_lcs = \
    mkcurves(star, fit.t, m.lmax, fit.sflux,
                    ncurves=m.ncurves, method="pca", negative=True, remove_y00=False, all_curves=True)
    
    np.savetxt("stored_pcas/pca30_y00_eigeny.txt", pca30_y00_eigeny)

else:
    try:
        pca30_y00_eigeny = np.loadtxt("stored_pcas/pca30_y00_eigeny.txt")
    except:
        print("PCA file not found, change read_file to True.")    

In [53]:
"""WILL NOT WORK!"""

# print("Making eigen values with tsvd, removing y00, and no negative maps")
# tsvd_norm_pos_eigeny, tsvd_norm_pos_evalues, tsvd_norm_pos_evectors, tsvd_norm_pos_ecurves, tsvd_norm_pos_lcs = \
# mkcurves(star, fit.t, m.lmax, fit.sflux,
#                 ncurves=m.ncurves, method="tsvd", negative=False, remove_y00=True)

'WILL NOT WORK!'

In [23]:
##OLD?

## Plotting Results

In [24]:
print("eigeny")
print(pca_norm_eigeny.shape)
print(tsvd_norm_eigeny.shape)
print(pca30_norm_eigeny.shape)
# print("ecurves")
# print(pca_norm_ecurves.shape)
# print(tsvd_norm_ecurves.shape)
# print("lcs")
# print(pca_norm_lcs.shape)
# print(tsvd_norm_lcs.shape)
# print("evectors")
# print(pca_norm_evectors.shape)
# print(tsvd_norm_evectors.shape)
# print("evalues")
# print(pca_norm_evalues.shape)
# print(tsvd_norm_evalues.shape)

eigeny
(15, 16)
(15, 16)
(30, 16)


In [25]:
# plt.plot(tsvd_norm_eigeny)
# plt.plot(tsvd_y00_eigeny)
# plt.plot(pca_norm_eigeny)
# plt.plot(pca_y00_eigeny)
# plt.plot(pca30_norm_eigeny)

In [ ]:
emaps(star, tsvd_norm_eigeny, "tsvd_norm_eigeny.png", proj="rect")

In [ ]:
emaps(star, tsvd_norm_eigeny2, "tsvd_norm_eigeny2.png", proj="rect")

In [ ]:
emaps(star, tsvd_y00_eigeny, "tsvd_y00_eigeny.png", proj="rect")

In [ ]:
emaps(star, tsvd_y00_eigeny2, "tsvd_y00_eigeny2.png", proj="rect")

In [ ]:
emaps(star, pca_norm_eigeny, "pca_norm_eigeny.png", proj="rect")

In [ ]:
emaps(star, pca_norm_eigeny2, "pca_norm_eigeny2.png", proj="rect")

In [ ]:
emaps(star, pca_y00_eigeny, "pca_y00_eigeny.png", proj="rect")

In [ ]:
emaps(star, pca_y00_eigeny2, "pca_y00_eigeny2.png", proj="rect")

In [ ]:
emaps(star, pca_norm_pos_eigeny, "pca_norm_pos_eigeny.png", proj="rect")

In [ ]:
emaps(star, pca_y00_pos_eigeny, "pca_y00_pos_eigeny.png", proj="rect")

In [ ]:
emaps(star, pca30_norm_eigeny, "pca30_norm_eigeny.png", proj='rect')

In [ ]:
emaps(star, pca30_y00_eigeny, "pca30_y00_eigeny.png",proj='rect')

In [23]:
def plot_eigen_flux(star, eigeny, outdir):

    ncurves, ny = eigeny.shape

    lmax = int(ny**0.5 - 1)
    thet = np.linspace(0, 360, ny)

    # ncols = np.int(np.sqrt(ncurves) // 1)
    # nrows = np.int(ncurves // ncols + (ncurves % ncols != 0))
    # npane = ncols * nrows

    fig, ax = plt.subplots(nrows=ncurves, ncols=1, figsize=(10, ncurves))

    for j in range(ncurves):
        star.map[1:,:] = 0

        yi = 1
        for l in range(1, lmax + 1):
            for mn in range(-l, l + 1):
                star.map[l, mn] = eigeny[j, yi]
                yi += 1
        
        ax[j].plot(star.map.flux(theta=thet).eval())
    
    fig.tight_layout()
    plt.savefig(f"plots/{outdir}")
    plt.close(fig)




In [24]:
plot_eigen_flux(star, tsvd_norm_eigeny, "tsvd_norm_eigeny_flux.png")

In [80]:
plot_eigen_flux(star, tsvd_norm_eigeny2, "tsvd_norm_eigeny_flux2.png")

In [79]:
plot_eigen_flux(star, tsvd_norm_eigeny3, "tsvd_norm_eigeny_flux3.png")

In [25]:
plot_eigen_flux(star, tsvd_y00_eigeny, "tsvd_y00_eigeny_flux.png")

In [26]:
plot_eigen_flux(star, pca_norm_eigeny, "pca_norm_eigeny_flux.png")

In [27]:
plot_eigen_flux(star, pca_y00_eigeny, "pca_y00_eigeny_flux.png")

In [28]:
plot_eigen_flux(star, pca_norm_pos_eigeny, "pca_norm_pos_eigeny_flux.png")

In [29]:
plot_eigen_flux(star, pca_y00_pos_eigeny, "pca_y00_pos_eigeny_flux.png")

In [30]:
plot_eigen_flux(star, pca30_norm_eigeny, "pca30_norm_eigeny_flux.png")

In [31]:
plot_eigen_flux(star, pca30_y00_eigeny, "pca30_y00_eigeny_flux.png")

## OLD?

In [ ]:
print("Running PCA to determine eigencurves.")
m.eigeny_i, m.evalues_i, m.evectors_i, m.ecurves_i, m.lcs_i = \
mkcurves(star, fit.t, m.lmax, fit.sflux,
                ncurves=m.ncurves, method=cfg.twod.pca)

Running PCA to determine eigencurves.
We are using the right mkcurves!
(30, 15937)
We are using the right pca!
(30, 15)


In [ ]:
print("Running PCA to determine eigencurves.")
m.eigeny_j, m.evalues_j, m.evectors_j, m.ecurves_j, m.lcs_j = \
mkcurves(star, fit.t, m.lmax, fit.sflux,
                ncurves=m.ncurves, method=cfg.twod.pca)

In [ ]:
#The lcs are the same!
print((m.lcs_i == m.lcs_j).all())

#The eigen maps are not!
print((m.eigeny_i == m.eigeny_j).all())

#The eigen maps are not!
print((m.ecurves_i == m.ecurves_j).all())

#The eigen maps are not!
print((m.evectors_i == m.evectors_j).all())

#The eigen maps are not!
print((m.evalues_i == m.evalues_j).all())

In [ ]:
#NEED TO CHANGE TO PCA IN CFG FILE

print("Running PCA to determine eigencurves.")
m.eigeny, m.evalues, m.evectors, m.ecurves, m.lcs = \
mkcurves(star, fit.t, m.lmax, fit.sflux,
                ncurves=m.ncurves, method="pca")

# m.eigeny2, m.evalues2, m.evectors2, m.ecurves2, m.lcs2 = \
# mkcurves(star, fit.t, m.lmax, fit.sflux,
#                 ncurves=m.ncurves, method="pca")

Running PCA to determine eigencurves.
We are using the right mkcurves!
(30, 15937)
We are using the right pca!
(30, 30)


In [ ]:
#The lcs are the same!
print((m.lcs == m.lcs2).all())

#The eigen maps are not!
print((m.eigeny == m.eigeny2).all())

#The eigen maps are not!
print((m.ecurves == m.ecurves2).all())

#The eigen maps are not!
print((m.evectors == m.evectors2).all())

#The eigen maps are not!
print((m.evalues == m.evalues2).all())

True
True
True
True
True


In [ ]:
print(f"The shape of lcs is {m.lcs.shape}\n")

print(f"The shape of ecurves (proj) is {m.ecurves.shape}\n")

# print(f"The shape of lcs with tsvd is {m.lcs_i.shape}\n")

# print(f"The shape of ecurves (proj) with tsvd is {m.ecurves_i.shape}\n")

The shape of lcs is (30, 15937)

The shape of ecurves (proj) is (30, 15937)



In [ ]:
print(m.evectors.shape)

(30, 30)


In [ ]:
print("Finding design matrix & rank of lcs")
lcs_A_T = m.lcs.T

lcs_R_T = np.empty((nsamples, lmax*2))

for k in range(nsamples):
    
    lcs_R_T[k] = [
        np.linalg.matrix_rank(lcs_A_T[:, : (l + 1) ** 2]) for l in range(1, lmax*2 + 1)
    ]
    # print(R[k])

lcs_R_T = np.median(lcs_R_T, axis=0)   

print(f"The new rank with lcs.T is {lcs_R_T}")

# lcs_A_T_i = m.lcs_i.T

# lcs_R_T_i = np.empty((nsamples, lmax*2))

# for k in range(nsamples):
    
#     lcs_R_T_i[k] = [
#         np.linalg.matrix_rank(lcs_A_T_i[:, : (l + 1) ** 2]) for l in range(1, lmax*2 + 1)
#     ]
#     # print(R[k])

# lcs_R_T_i = np.median(lcs_R_T_i, axis=0)   

# print(f"The new rank with lcs.T with tsvd is {lcs_R_T_i}")

Finding design matrix & rank of lcs
The new rank with lcs.T is [2. 3. 5. 5. 5. 5.]


In [ ]:
print("Finding design matrix & rank of ecurves")

evect_A_T = m.ecurves.T

evect_R_T = np.empty((nsamples, lmax))

for k in range(nsamples):
    
    evect_R_T[k] = [
        np.linalg.matrix_rank(evect_A_T[:, : (l + 1) ** 2]) for l in range(1, lmax + 1)
    ]
    # print(R[k])

print(evect_R_T.shape)

evect_R_T = np.median(evect_R_T, axis=0)   

print(f"The new rank with ecurves.T is {evect_R_T}")

# evect_A_T_i = m.ecurves_i.T

# evect_R_T_i = np.empty((nsamples, lmax))

# for k in range(nsamples):
    
#     evect_R_T_i[k] = [
#         np.linalg.matrix_rank(evect_A_T_i[:, : (l + 1) ** 2]) for l in range(1, lmax + 1)
#     ]
#     # print(R[k])

# evect_R_T_i = np.median(evect_R_T_i, axis=0)   

# print(f"The new rank with ecurves.T with tsvd is {evect_R_T_i}")

Finding design matrix & rank of ecurves
(100, 3)
The new rank with ecurves.T is [4. 4. 4.]


In [ ]:
print("Finding design matrix & rank of eigeny")
print(f"It has shape {m.eigeny.shape}")

eigeny_A = m.eigeny #m.eigeny.design_matrix()

eig_R = np.empty((nsamples, lmax))

for k in range(nsamples):

    eig_R[k] = [
        np.linalg.matrix_rank(eigeny_A[:, : (l + 1) ** 2]) for l in range(1, lmax + 1)
    ]
    # print(R[k])

eig_R = np.median(eig_R, axis=0)

print(f"The new rank with eigeny is {eig_R}")

eigeny_A_T = m.eigeny.T #m.eigeny.design_matrix()

eig_R_T = np.empty((nsamples, lmax))

for k in range(nsamples):

    eig_R_T[k] = [
        np.linalg.matrix_rank(eigeny_A_T[:, : (l + 1) ** 2]) for l in range(1, lmax + 1)
    ]
    # print(R[k])

eig_R_T = np.median(eig_R_T, axis=0)

print(f"The new rank with eigeny.T is {eig_R_T}")

# eigeny_A_T_i = m.eigeny_i.T #m.eigeny.design_matrix()

# eig_R_T_i = np.empty((nsamples, lmax))

# for k in range(nsamples):

#     eig_R_T_i[k] = [
#         np.linalg.matrix_rank(eigeny_A_T_i[:, : (l + 1) ** 2]) for l in range(1, lmax + 1)
#     ]
# # print(R[k])

# eig_R_T_i = np.median(eig_R_T_i, axis=0)

# print(f"The new rank with eigeny.T with tsvd is {eig_R_T_i}")

Finding design matrix & rank of eigeny
It has shape (15, 16)
The new rank with eigeny is [ 4.  9. 11.]
The new rank with eigeny.T is [ 4.  8. 11.]


In [ ]:
plots.lightcurves(fit.t, m.lcs, outdir, "lcs_light_curves")

In [ ]:
# plots look like they are doubling up!

plots.emaps(star, m.eigeny, outdir, proj='rect')

In [ ]:
eigen_flux = []
ncurves, ny = m.eigeny.shape

lmax = int(ny**0.5 - 1)
thet = np.linspace(0, 360, ny)

# ncols = np.int(np.sqrt(ncurves) // 1)
# nrows = np.int(ncurves // ncols + (ncurves % ncols != 0))
# npane = ncols * nrows

for j in range(ncurves):
    star.map[1:,:] = 0

    # xloc = j %  ncols
    # yloc = j // ncols
    
    yi = 1
    for l in range(1, lmax + 1):
        for mn in range(-l, l + 1):
            star.map[l, mn] = m.eigeny[j, yi]
            yi += 1

    print(f"The {j} index eigen map's flux is {star.map.flux(theta=thet).eval()}")
    eigen_flux.append(star.map.flux(theta=thet).eval())

The 0 index eigen map's flux is [ 2.60439549  2.4656796   2.07352286  1.49574607  0.83226421  0.19780226
 -0.29794381 -0.56926893 -0.56926893 -0.29794381  0.19780226  0.83226421
  1.49574607  2.07352286  2.4656796   2.60439549]
The 1 index eigen map's flux is [ 1.          1.6525521   2.19227202  2.52583727  2.59557139  2.38941673
  1.94301929  1.33356525  0.66643475  0.05698071 -0.38941673 -0.59557139
 -0.52583727 -0.19227202  0.3474479   1.        ]
The 2 index eigen map's flux is [1.61918332 1.41429219 0.93520708 0.49896808 0.39426755 0.69040834
 1.19144026 1.56582484 1.56582484 1.19144026 0.69040834 0.39426755
 0.49896808 0.93520708 1.41429219 1.61918332]
The 3 index eigen map's flux is [1.         0.53978926 0.38411782 0.6359995  1.12875444 1.53630757
 1.58896518 1.25188169 0.74811831 0.41103482 0.46369243 0.87124556
 1.3640005  1.61588218 1.46021074 1.        ]
The 4 index eigen map's flux is [1.26064866 1.26064866 1.26064866 1.26064866 1.26064866 1.26064866
 1.26064866 1.2606486

In [ ]:


test = np.random.rand(4,2)



In [ ]:
test

array([[0.21816324, 0.19414291],
       [0.52985809, 0.46880417],
       [0.61073674, 0.03797679],
       [0.75341679, 0.17138988]])

In [ ]:
test.T

array([[0.21816324, 0.52985809, 0.61073674, 0.75341679],
       [0.19414291, 0.46880417, 0.03797679, 0.17138988]])